# Quality Model App

## Requirements

In [ ]:
%pip install ipywidgets=7.7.3

## App module and function

In [ ]:
from quality_model_widgets import QualityModelWidgets, create_input_data_dict

## Specify Data Source and Output File

In [ ]:
input_dir = '../dataset/Tile_Generator_Genetic_Algo_V1_16x16-2023-23-2--16-01-20'
output_path = 'output.json'

## Run on Data Source

In [ ]:
data_dict = create_input_data_dict(input_dir)

## Widgets (Start the App)

In [ ]:
qualityModelWidgets = QualityModelWidgets(data_dict, output_path)
qualityModelWidgets.start()